In [2]:
from robotic import ry
import numpy as np
import time

## Part a) Adding a constraint to simulation

In [191]:
C = ry.Config()
C.addFile(ry.raiPath('scenarios/pandaSingle.g'))
C.addFrame('way1'). setShape(ry.ST.marker, [.1]) .setPosition([.4, .2, 1.])
C.addFrame('way3'). setShape(ry.ST.marker, [.1]) .setPosition([-.4, .2, 1.])

C.addFrame('box') \
    .setPosition([-.25,.1,1.]) \
    .setShape(ry.ST.ssBox, size=[.06,.6,.6,.005]) \
    .setColor([1,.5,0]) \
    .setContact(True)

C.view()
qHome = C.getJointState()

First, solve the task without collision constraint

In [192]:
C.setJointState(qHome)
komo = ry.KOMO(C, 5, 10, 1, True)
komo.addControlObjective([], 0, 1e-1)
komo.addControlObjective([], 1, 1e0)
komo.addObjective([1], ry.FS.positionDiff, ['l_gripper', 'way1'], ry.OT.eq, [1e1])
komo.addObjective([3], ry.FS.positionDiff, ['l_gripper', 'way3'], ry.OT.eq, [1e1])
komo.addObjective([5], ry.FS.positionDiff, ['l_gripper', 'way1'], ry.OT.eq, [1e1])

ret = ry.NLP_Solver(komo.nlp(), verbose=0 ) .solve()
print(ret)
q = komo.getPath()
print(q)

for t in range(q.shape[0]):
    C.setJointState(q[t])
    C.view(False, f'waypoint {t}')
    time.sleep(0.1)

{ time: 0.011382, evals: 15, done: 1, feasible: 1, sos: 1.88428, f: 0, ineq: 0, eq: 0.00137863 }
[[-3.70241485e-02 -4.54630799e-01 -3.79055723e-02 -2.00581393e+00
  -9.40222567e-03  2.01357904e+00 -5.00007547e-01]
 [-7.40890682e-02 -4.09212132e-01 -7.58475761e-02 -2.01162969e+00
  -1.88090439e-02  2.02716981e+00 -5.00014823e-01]
 [-1.11235483e-01 -3.63694694e-01 -1.13862496e-01 -2.01744919e+00
  -2.82251617e-02  2.04078395e+00 -5.00021561e-01]
 [-1.48504025e-01 -3.18029492e-01 -1.51986923e-01 -2.02327463e+00
  -3.76555145e-02  2.05443291e+00 -5.00027500e-01]
 [-1.85935203e-01 -2.72167968e-01 -1.90257620e-01 -2.02910847e+00
  -4.71053777e-02  2.06812786e+00 -5.00032393e-01]
 [-2.23569382e-01 -2.26062096e-01 -2.28711590e-01 -2.03495364e+00
  -5.65804760e-02  2.08187968e+00 -5.00036008e-01]
 [-2.61446772e-01 -1.79664437e-01 -2.67386152e-01 -2.04081351e+00
  -6.60870876e-02  2.09569887e+00 -5.00038130e-01]
 [-2.99607454e-01 -1.32928147e-01 -3.06319041e-01 -2.04669197e+00
  -7.56321424e-02 

Then, we add collision constraint with "accumulatedCollisions" feature

In [193]:
del komo
del C

In [194]:
C = ry.Config()
C.addFile(ry.raiPath('scenarios/pandaSingle.g'))
C.addFrame('way1'). setShape(ry.ST.marker, [.1]) .setPosition([.4, .2, 1.])
C.addFrame('way3'). setShape(ry.ST.marker, [.1]) .setPosition([-.4, .2, 1.])

C.addFrame('box') \
    .setPosition([-.25,.1,1.]) \
    .setShape(ry.ST.ssBox, size=[.06,.6,.6,.005]) \
    .setColor([1,.5,0]) \
    .setContact(True)

C.view()
qHome = C.getJointState()

But we see that there is a jump in the motion while passing the obstacle

Here we will see the importance of the scale of the constraint (give values 0, 100 and 1 (which is default one) relatively). Then put 0.1 (1e-1) for perfect case. Do not forget to Del K and C or restart.

In [195]:
C.setJointState(qHome)
komo = ry.KOMO(C, 5, 10, 1, True)
komo.addControlObjective([], 0, 1e-1)
komo.addControlObjective([], 1, 1e0)
komo.addObjective([], ry.FS.accumulatedCollisions, [], ry.OT.eq,[1e1]);
komo.addObjective([1], ry.FS.positionDiff, ['l_gripper', 'way1'], ry.OT.eq, [1e1])
komo.addObjective([3], ry.FS.positionDiff, ['l_gripper', 'way3'], ry.OT.eq, [1e1])
komo.addObjective([5], ry.FS.positionDiff, ['l_gripper', 'way1'], ry.OT.eq, [1e1])

ret = ry.NLP_Solver(komo.nlp(), verbose=0 ) .solve()
print(ret)
q = komo.getPath()
print(q)

for t in range(q.shape[0]):
    C.setJointState(q[t])
    C.view(False, f'waypoint {t}')
    time.sleep(0.1)

{ time: 1.0082, evals: 728, done: 1, feasible: 0, sos: 3.4479, f: 0, ineq: 0, eq: 2.40483 }
[[-0.039359   -0.45492098 -0.04642391 -2.00770361  0.03575031  2.01368697
  -0.50155203]
 [-0.07875925 -0.40979932 -0.0928969  -2.01542051  0.07155876  2.02739257
  -0.5031021 ]
 [-0.11824204 -0.36459234 -0.1394681  -2.02316398  0.10748346  2.04113542
  -0.50464824]
 [-0.15784872 -0.31925725 -0.18618672 -2.03094732  0.1435825   2.05493419
  -0.50618849]
 [-0.19762073 -0.27375111 -0.23310207 -2.0387838   0.17991389  2.06880757
  -0.50772083]
 [-0.23759964 -0.22803081 -0.28026367 -2.04668673  0.21653555  2.08277429
  -0.50924327]
 [-0.27782717 -0.18205297 -0.3277212  -2.05466937  0.25350528  2.09685314
  -0.51075376]
 [-0.31834522 -0.13577391 -0.37552462 -2.06274499  0.29088075  2.11106296
  -0.51225022]
 [-0.35919592 -0.08914962 -0.42372415 -2.07092685  0.3287194   2.12542265
  -0.51373054]
 [-0.40042163 -0.04213569 -0.47237034 -2.07922819  0.36707846  2.13995117
  -0.51519258]
 [-0.37276376 -0.0

komo.addObjective([], ry.FS.accumulatedCollisions, [], ry.OT.eq);

    komo.addObjective(...): This is calling a method named addObjective on the komo object, which is an instance of a KOMO optimizer.

    []: The first argument [] specifies the time intervals over which this objective applies. In this case, [] means it applies to all time intervals. In other words, this objective is valid for the entire duration of the optimization.

    ry.FS.accumulatedCollisions: The second argument ry.FS.accumulatedCollisions is specifying the type of feature to use for this objective. In this case, it's using a feature related to accumulated collisions. This means the objective will aim to minimize the total amount of collisions that occur throughout the motion.

    []: The third argument [] specifies the frames associated with this feature. Since ry.FS.accumulatedCollisions doesn't require specific frame associations, this argument is left empty.

    ry.OT.eq: The fourth argument ry.OT.eq specifies the type of objective. Here, it's an equality constraint. This means the optimizer will aim to make the value of the accumulated collisions equal to a certain target value (which is default value zero in here).

Putting it all together, this line is instructing the optimizer to minimize the total amount of accumulated collisions over the entire duration of the motion, treating it as an equality constraint. This is a way to prioritize collision-free or low-collision solutions.

##  Part b) Possible State Solution by KOMO

In [150]:
del komo
del C

In [3]:
C = ry.Config()
C.addFile(ry.raiPath('scenarios/pandaSingle.g'))
C.addFrame('way3'). setShape(ry.ST.marker, [.1]) .setPosition([-.4, .2, 1.])

C.addFrame('box') \
    .setPosition([-.25,.1,1.3]) \
    .setShape(ry.ST.ssBox, size=[.06,.7,.5,.005]) \
    .setColor([1,.5,0]) \
    .setContact(True)

C.view()
qHome = C.getJointState()

In [4]:
C.setJointState(qHome)
komo = ry.KOMO(C, 1, 10, 2, True)
komo.addControlObjective([], 0, 1e-1) 
komo.addControlObjective([], 2, 1e0)
komo.addObjective([], ry.FS.accumulatedCollisions, [], ry.OT.eq,[1e1]);
komo.addObjective([1], ry.FS.positionDiff, ['l_gripper', 'way3'], ry.OT.eq, [1e1])
komo.addObjective([1], ry.FS.jointState, [], ry.OT.eq, [1e1], [], order=1)

ret = ry.NLP_Solver(komo.nlp(), verbose=0 ) .solve()
print(ret)
q = komo.getPath()
print('size of path:', q.shape)

for t in range(q.shape[0]):
    C.setJointState(q[t])
    C.view(False, f'waypoint {t}')
    time.sleep(.1)

{ time: 0.332288, evals: 538, done: 1, feasible: 1, sos: 59.8798, f: 0, ineq: 0, eq: 0.0316176 }
size of path: (10, 7)


## Possible State Solution by RRT

In [5]:
# that's the goal configuration
qT = komo.getPath()[-1]
C.setJointState(qT)
C.view(False, "IK solution")

0

In [6]:
#define a path finding problem
rrt = ry.PathFinder()
rrt.setProblem(C, [qHome], [qT])

ret = rrt.solve()
print(ret)
path = ret.x

{ time: 0.011427, evals: 60, done: 1, feasible: 1, sos: -1, f: -1, ineq: -1, eq: -1 }


In [7]:
# display the path
for t in range(0, path.shape[0]-1):
    C.setJointState(path[t])
    C.view()
    time.sleep(.1)

## RRT Solution on BotOp 

In [8]:
# run the path with botop
C.setJointState(qHome)
ry.params_add({'botsim/verbose': 1., 'physx/motorKp': 10000., 'physx/motorKd': 1000.})
bot = ry.BotOp(C, False)
bot.home(C)

In [9]:
bot.moveAutoTimed(path, 1., 1.)
while bot.getTimeToEnd()>0:
    bot.sync(C, .1)

0.184748 1


In [10]:
del komo
del rrt
del bot
del C

-- bot.cpp:~BotOp:106(0) shutting down BotOp...
-- simulation.cpp:~BotThreadedSim:56(0) shutting down SimThread
-- simulation.cpp:~Simulation:148(0) shutting down Simulation


Lets give impossible state

##  Part c) Impossible State Solution by KOMO

In [11]:
C = ry.Config()
C.addFile(ry.raiPath('scenarios/pandaSingle.g'))
C.addFrame('way3'). setShape(ry.ST.marker, [.1]) .setPosition([-.4, .2, 1.])

C.addFrame('box') \
    .setPosition([-.25,.1,1.]) \
    .setShape(ry.ST.ssBox, size=[.06,.7,.7,.005]) \
    .setColor([1,.5,0]) \
    .setContact(True)

C.view()
qHome = C.getJointState()

In [12]:
C.setJointState(qHome)
komo = ry.KOMO(C, 1, 10, 2, True)
komo.addControlObjective([], 0, 1e-1) 
komo.addControlObjective([], 2, 1e0)
komo.addObjective([], ry.FS.accumulatedCollisions, [], ry.OT.eq,[1e-1]);
komo.addObjective([1], ry.FS.positionDiff, ['l_gripper', 'way3'], ry.OT.eq, [1e1])
komo.addObjective([1], ry.FS.jointState, [], ry.OT.eq, [1e1], [], order=1)

ret = ry.NLP_Solver(komo.nlp(), verbose=0 ) .solve()
print(ret)
q = komo.getPath()
print('size of path:', q.shape)

for t in range(q.shape[0]):
    C.setJointState(q[t])
    C.view(False, f'waypoint {t}')
    time.sleep(.1)

{ time: 0.058206, evals: 57, done: 1, feasible: 1, sos: 7.82928, f: 0, ineq: 0, eq: 0.176068 }
size of path: (10, 7)


## Impossible State Solution by RRT

In [13]:
# that's the goal configuration
qT = komo.getPath()[-1]
C.setJointState(qT)
C.view(False, "IK solution")

0

In [14]:
#define a path finding problem
rrt = ry.PathFinder()
rrt.setProblem(C, [qHome], [qT])

ret = rrt.solve()
print(ret)
path = ret.x

-- RRT_PathFinder.cpp:RRT_SingleTree:55(0) rooting RRT with infeasible start configuration -- that's likely to fail: query is:
query: h_goal: 0 g_coll: 0.191575 isGoal: 1 isFeasible: 0
{ time: 0.99164, evals: 5001, done: 1, feasible: 0, sos: -1, f: -1, ineq: -1, eq: -1 }


So we will get an error since there is no feasible solution

In [16]:
# display the path
for t in range(0, path.shape[0]-1):
    C.setJointState(path[t])
    C.view()
    time.sleep(.1)

IndexError: tuple index out of range